# Projecting a function onto a finite element space

In this notebook we show how to define a scalar FEM space over a given (discretized) domain, how to plot some basis functions, assemble a mass matrix and compute the L2 projection of a given function 

## Step 1 : define the domain and discretize it

This is similar to what is done in other examples

In [ ]:
import numpy as np

from sympde.topology import Square, PolarMapping
from sympde.utilities.utils import plot_domain

from psydac.api.tests.build_domain import build_pretzel

# Define the topological geometry for each patch
rmin, rmax = 0.3, 1.

# First quarter annulus
domain_log_1 = Square('A_1', bounds1=(0., 1.), bounds2=(0., np.pi/2))
F_1 = PolarMapping('F_1', dim=2, c1=0., c2=0., rmin=rmin, rmax=rmax)
Omega_1 = F_1(domain_log_1)

# Second quarter annulus
domain_log_2 = Square('A_2', bounds1=(0., 1.), bounds2=(np.pi, 3/2 * np.pi))
F_2 = PolarMapping('F_2', dim=2, c1=rmin+rmax, c2=0., rmin=rmax, rmax=rmin)
Omega_2 = F_2(domain_log_2)

# Join the patches

# [remark] this could be an opportunity to describe how the interfaces of a domain are numbered
from sympde import Domain
connectivity = [((0,1,-1),(1,1,-1), 1)]
patches = [Omega_1, Omega_2]
Omega = Domain.join(patches, connectivity, 'domain')

# Example of a complex multi-patch domain
# Omega = build_pretzel()

# Simple visualization of the topological domain
plot_domain(Omega, draw=False, isolines=True)

## Step 2 : define a FEM space and plot some basis functions

[this could actually be a good opportunity to present how the indices work in psydac arrays]

In [ ]:
from sympde.topology    import ScalarFunctionSpace

from psydac.linalg.utilities import array_to_psydac
from psydac.api.discretization import discretize
from psydac.fem.basic import FemField
from psydac.fem.plotting_utilities import plot_field_2d as plot_field

# the kind argument also specifies the pushforward defining the FEM space on the physical domain
# kind=None results in a (scalar, vector?) space defined by a simple change of variable (0-form pushforward)
V   = ScalarFunctionSpace('V', Omega, kind=None)

ncells = [10, 10]
degree = [2, 2]

# [remark] for multipatch domains the periodic flag is a priori unrelevant, should raise an error / warning 
# periodic = [ False, True]
periodic = [False, False] 

Omega_h = discretize(Omega, ncells=ncells, periodic=periodic)
Vh = discretize(V, Omega_h, degree=degree)

npatches = len(Vh.patch_spaces)
for k in range(npatches):
    knots = Vh.patch_spaces[k].knots[0]
    print(f'Spline knots (logical coords) along axis 0 in patch {k}: \n {knots}')

# plot some basis function from the space 
# [remark] here I'm using numpy arrays, but one could illustrate the psydac indexing
bf_c = np.zeros(Vh.nbasis)
bf_c[Vh.nbasis//4] = 1
bf_c[Vh.nbasis//4+6] = 1
bf = FemField(Vh, coeffs=array_to_psydac(bf_c, Vh.coeff_space))

plot_field(fem_field=bf, Vh=Vh, space_kind=None, domain=Omega, title='two basis functions', hide_plot=False)



## Step 3 : compute the mass matrix

we assemble the mass matrix and visualize it

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm, colors

from sympde.topology    import elements_of
from sympde.expr.expr   import BilinearForm
from sympde.expr.expr   import integral              

from psydac.api.settings import PSYDAC_BACKENDS

backend_language = 'python'
backend = PSYDAC_BACKENDS[backend_language]

# domain = V.domain  # = Omega
u, v = elements_of(V, names='u, v')
a = BilinearForm((u, v), integral(Omega, u * v))
ah = discretize(a, Omega_h, [Vh, Vh], backend=backend)

M = ah.assemble()  # Mass matrix in stencil format (linear operator)

# visualize the mass matrix by plotting its numpy conversion
mat = M.toarray()

#----------------
fig,ax = plt.subplots(1,1)
ax.set_title(f"Mass matrix M on the domain with {npatches} patch(es)")
im = ax.matshow(mat, norm=colors.LogNorm(), cmap='hot_r')
cb = fig.colorbar(im, ax=ax)
fig.show()

## Step 4 : compute the moments of a given target function and its L2 projection

we invert the mass matrix on the moments of a given function f

In [ ]:
from psydac.linalg.solvers import inverse
from psydac.fem.projectors import get_dual_dofs

x,y     = Omega.coordinates
ref_f   = (x-1/2)**2 + y**2
tilde_f = get_dual_dofs(Vh=Vh, f=ref_f, domain_h=Omega_h, backend_language=backend_language)

inv_M = inverse(M, solver='cg')

f_h = inv_M @ tilde_f

plot_field(stencil_coeffs=f_h, Vh=Vh, space_kind='h1', domain=Omega, title='f_h: L2 projection of f', hide_plot=False)